## Visualization of Schism results

In [ ]:
#to use the full width of the browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.tri as tri
from netCDF4 import Dataset
from scipy.io import loadmat
import pandas as pd
from scipy.interpolate import griddata
from tqdm import tqdm

In [ ]:
from Poseidon.utils import *


In [ ]:
from matplotlib import animation,rc

rc('animation',html='html5')


In [ ]:
PATH='/home/critechproc/STORAGE/SCHISM/EUROP/'

In [ ]:
#g=loadmat('/home/critechproc/STORAGE/SCHISM/test1/nodes.mat')
#g['nodes'][:].shape

In [ ]:
grid=PATH+'hgrid.gr3'

In [ ]:
#extract number of elements, number of nodes
ne,nn = pd.read_csv(grid,header=0,skiprows=1,nrows=0,delim_whitespace=True)

In [ ]:
ne=int(ne)
nn=int(nn)

In [ ]:
#read lon,lat,depth for all nodes
q=pd.read_csv(grid,skiprows=2,header=None,delim_whitespace=True,engine='python',nrows=nn,names=['id','x','y','z'])

In [ ]:
q=q.set_index(['id'])

In [ ]:
q.head()

In [ ]:
e = pd.read_csv(grid,skiprows=nn+2,header=None,delim_whitespace=True,engine='python',nrows=ne,names=['id','nv','a','b','c'])

In [ ]:
e=e.set_index(['id'])

In [ ]:
r = Dataset(PATH+'outputs/schism_history.nc')

In [ ]:
r.variables.keys()

In [ ]:
U = r.variables['davx'][:]
V = r.variables['davy'][:]

In [ ]:
h = r.variables['WATLEV'][:]

In [ ]:
h

In [ ]:
time = r.variables['ocean_time_str'][:]

In [ ]:
t = [''.join(x) for x in time] #join strings in readable form

In [ ]:
len(t) # time steps

In [ ]:
t

## plot

In [ ]:
triangles = e.loc[:,['a','b','c']].values

In [ ]:
triangles

In [ ]:
triangles.max()

In [ ]:
tri3 = triangles-1 # adjust for python

In [ ]:
x = q.loc[:,['x']].values.flatten()
y = q.loc[:,['y']].values.flatten()

In [ ]:
x.shape,y.shape

In [ ]:
z = q.loc[:,['z']].values.flatten() #depth

In [ ]:
minlon=-35. # lat/lon window
maxlon=43.
minlat=24.5
maxlat=76.5

In [ ]:
plt.figure(figsize=(12,10))
plt.gca().set_aspect('equal')
plt.tricontourf(x, y, tri3, h[10,:], 50, )
plt.colorbar()
plt.title('Contour plot')
plt.xlabel('Longitude (degrees)')
plt.ylabel('Latitude (degrees)')
plt.show()

## grid plot

In [ ]:
plt.figure(figsize=(12,10))
plt.triplot(x,y,tri3,'go-', lw=.5, markersize=5)
plt.show()

## vector plot

In [ ]:
plt.figure(figsize=(12,10))
plt.gca().set_aspect('equal')
plt.quiver(x[::10], y[::10], U[0,::10], V[0,::10], angles='xy', scale_units='xy', scale=5)
plt.title('Vector plot')
plt.xlabel('Longitude (degrees)')
plt.ylabel('Latitude (degrees)')
plt.show()

## anim

In [ ]:
def anim(z,t,title=None):
    fig = plt.figure(figsize=(12,8))
    ax = fig.gca()
    ax.set_aspect('equal')
    v=np.linspace(z.min(),z.max(),10,endpoint=True)
    ims = []
    for i in tqdm(range(len(t))):
        im = ax.tricontourf(x, y, tri3, z[i,:], v, vmin=z.min(), vmax=z.max())
#        im = ax.contourf(x,y,z[i,:,:],v,vmin=v1,vmax=v2,latlon=True)
        add_arts = im.collections
        text = 'time={}'.format(t[i])
        #te = ax.text(90, 90, text)
        an = ax.annotate(text, xy=(0.05, 0.95), xycoords='axes fraction')
        ims.append(add_arts + [an])
    ax.set_title(title)    
#cbar_ax = fig.add_axes([0.05, 0.05, 0.85, 0.05])    
    cbar = fig.colorbar(im,ticks=v,orientation='vertical', extend='both')#,fraction=0.046, pad=0.04)
#plt.colorbar()
    return animation.ArtistAnimation(fig, ims, interval=200, blit=False,repeat=False)

In [ ]:
anim(h,t,title='Storm Surge')

In [ ]:
#anim(U,t,title='U - velocity')